In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [2]:
!pip install -q -U deepeval --use-deprecated=legacy-resolver

ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you'll have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you'll have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you'll have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you'll have pyarrow 16.1.0 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you'll have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you'll have requests-toolbelt 1.0.0 which is incompatible.
opentelemetry-exporter-otlp 1.25.0 requires opentelemetry-exporter-otlp-proto-grpc==1.25.0, but you'll have opentelemetry-exporter-otlp-proto-grpc 1.24.0 which is incompatible.
opentele

In [3]:
!pip install -q -U pydantic
!pip install -q -U lm-format-enforcer

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.10.1 which is incompatible.
langchain-core 0.3.21 requires packaging<25,>=23.2, but you have packaging 21.3 which is incompatible.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.1 which is incompatible.


Create a dataset with the pytei files (__init__.py, pytei.py, dpytei.py) and upload the dataset named pytei-files

In [4]:
import sys
sys.path.append( "/kaggle/input/pytei-files" )
from pytei import Injector

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Put your hugging face token below

In [6]:
token = ""

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
from deepeval.models.base_model import DeepEvalBaseLLM
from typing import List
from pydantic import BaseModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import (
    build_transformers_prefix_allowed_tokens_fn,
)
import transformers
from transformers import pipeline
import json

class GPT2(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        model = self.load_model()
        pipeline = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            use_cache=True,
            device_map="auto",
            max_new_tokens=100,
            do_sample=True,
            top_k=5,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )

        # Create parser required for JSON confinement using lmformatenforcer
        parser = JsonSchemaParser(schema.schema())
        prefix_function = build_transformers_prefix_allowed_tokens_fn(
            pipeline.tokenizer, parser
        )

        # Output and load valid JSON
        output_dict = pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)
        output = output_dict[0]["generated_text"][len(prompt) :]
        json_result = json.loads(output)

        # Return valid JSON object according to the schema DeepEval supplied
        return schema(**json_result)

    async def a_generate(self, prompt: str, schema) -> BaseModel:
        return self.generate(prompt, schema)

    # This is optional.
    def batch_generate(self, promtps: List[str]) -> List[str]:
        model = self.load_model()
        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer(promtps, return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)

    def get_model_name(self):
        return "GPT2"

In [9]:
import re
from torch import nn
def print_named_params(model: nn.Module) -> None:
    for name, param in model.named_parameters():
        print(f"{name}: {param.shape}")
def output_targets(model: nn.Module, file: str, regex: str) -> None:
    with open(f'{file}', 'w') as f:
        for name, param in model.named_parameters():
            if (re.match(regex, name)):
                f.write(f"{name}\n")

# Define regex patterns for each phase
regex_patterns = {
    "embedding": r"^model\.embed_tokens\..*",
    "attention": r"^model\.layers\.\d+\.self_attn\..*",
    "feed_forward": r"^model\.layers\.\d+\.mlp\..*",
    "normalization": r"^model\.(layers\.\d+\.(input_layernorm|post_attention_layernorm)|norm)\..*",
    "output": r"^lm_head\..*",
}

phases = list(regex_patterns.keys())

def output_parameters_by_phase(model: nn.Module, phase: str, phase_filename: str):
    
    # Regex for the specified phase
    phase_pattern = regex_patterns[phase]
    
    # Collect parameters for the specified phase
    phase_params = []
    for name, param in model.named_parameters():
        if re.match(phase_pattern, name):
            phase_params.append(name)
    
    # Write the specified phase's parameters to a file
    with open(phase_filename, 'w') as f:
        for name in phase_params:
            f.write(f"{name}\n")
    
    print(f"Parameters for phase '{phase}' have been written to {phase_filename}")


You will get an error 1-2 times when running this, whenever this happens, go to "Run" at the top and restart the kernel (NOT the session)

There is some load up time for the first iteration to download the model, but everything after should be fast

The code will throw an error everytime layer switches so you need to be at the computer to switch layers. Each computation for a fixed layer and probability should take about 15 minutes. Whenever you restart the session you need to save your computer or the files to your computer. It would be best if you could save the output files to your local computer and just send them. They are named "mistral_output_{x}_{???}".

In [ ]:
from deepeval.benchmarks import MMLU
from deepeval.benchmarks.tasks import MMLUTask
import time
import gc    
import logging

probabilities = [7, 6, 5, 4, 3, 2, 1]

model_name = 'mistralai/Mistral-7B-Instruct-v0.3'
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token, trust_remote_code=True)

benchmark = MMLU(
    tasks=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ASTRONOMY],
    n_shots=1
)

for phase in phases[4:]:

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        #quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        token=token,
    )

    
    target_file = f"mistral_target_{phase}"
    output_file = f"mistral_output_{phase}"
    output_parameters_by_phase(model, phase, target_file)
    
    for prob in probabilities:

        print(f"Running with injections at {phase} with prob: {prob}\n")
    
        injector = Injector(target_file, p = pow(10, -prob), device = device, dtype = torch.float16, verbose = True)
        injector.inject_values(model)
        mistral = GPT2(model, tokenizer)
        benchmark.evaluate(model=mistral)
        
        with open(output_file, 'a') as f:
            
            f.write(f'Injected at {phase} with prob: {prob} | Score: {benchmark.overall_score}\n')

        print(f'Injected at {phase} with prob: {prob} | Score: {benchmark.overall_score}\n')


README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

Filter:   0%|          | 0/164 [00:00<?, ? examples/s]

Filter:   0%|          | 0/164 [00:00<?, ? examples/s]

...

Processing astronomy: 100%|██████████| 152/152 [10:07<00:00,  4.00s/it]


MMLU Task Accuracy (task=astronomy): 0.5526315789473685
Overall MMLU Accuracy: 0.5634920634920635
Injected at output with prob: 5 | Score: 0.5634920634920635

Running with injections at output with prob: 4

Injector initialized.
Error probability: 0.0001
Data type: torch.float16
Error model: bit


Processing high_school_computer_science: 100%|██████████| 100/100 [06:16<00:00,  3.76s/it]


MMLU Task Accuracy (task=high_school_computer_science): 0.53


Processing astronomy:  27%|██▋       | 41/152 [02:20<06:21,  3.43s/it]